In [8]:
import pickle
import os
import sys
sys.path.append("../")
from od.social.group import SocialGroup
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,NET_TIMEOUT_SECONDS,SUMO_TOTAL_SECONDS
from od.env.station import BS_PRESET
from od.social.manager import  DynamicSocialGroupBehaviour,QoSLevel
import od.misc.statistic as oms

In [9]:
statistics = [[],[]]


In [11]:
print(os.getcwd())
for i,v in enumerate([ResourceAllocatorType.NOMA_APR,ResourceAllocatorType.NOMA_OPT]):
        for s in range(1):
                interest_config = InterestConfig(
                        DynamicSocialGroupBehaviour.MAX_N_MEMBER,
                        20,
                        False,
                        v,
                        True,
                        1.4,
                        s+1
                )
                with open("../data/POST_REGNA/{}/object.pickle".format(interest_config.folder()), "rb") as file:
                        statistics[i].append(pickle.load(file))


/home/OD/repos/ProjectSumo/ipynb


In [14]:
for stats in statistics:
    print([str(stat.interest_config) for stat in stats])

['MAX_N_MEMBER(20)-nQoS-yRSU-NOMA_APR-D1.4-S1']
['MAX_N_MEMBER(20)-nQoS-yRSU-NOMA_OPT-D1.4-S1']


In [15]:

BS_CONFIG =  {
        name:bs_config 
        for name,bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA  or 
            (
                interest_config.req_rsu and 
                bs_config["type"] == BaseStationType.UMI
            )
        )
}

class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial
        
class GV:
    NET_STATION_CONTROLLER = [ 
        FakeBaseStation(
            name,
            BS_CONFIG[name]["pos"],
            BS_CONFIG[name]["type"] ,
            i
        )   for i,name in enumerate(BS_CONFIG)
    ]
    class STATISTIC:
        def Log(data):
            pass
        def Doc(data):
            pass
    class RESULT:
        def Log(data):
            pass
        def Doc(data):
            print(data)
oms.GV = GV

In [16]:
GV.NET_STATION_CONTROLLER = GV.NET_STATION_CONTROLLER[:2]
GV.NET_STATION_CONTROLLER

[UMA-1, UMA-2]

In [17]:
class Tabber(str):
    def __init__(self):
        self.tabs = 0
    def __str__(self):
        return "".join(["\t" for _ in range(self.tabs)])

    def enter(self):
        self.tabs +=1
    def exit(self):
        self.tabs -=1

def ppp():
    global statistics,ots,c
    tabber = Tabber()
    for i,stats in enumerate(statistics):
        for stat in stats:
            print(str(tabber)+str(stat.interest_config))
            tabber.enter()
            for sg in sorted(stat.social_group,key=lambda x: x.gid):
                src_ot = 0
                bs_ot = [0 for i in GV.NET_STATION_CONTROLLER]
                members = set()
                if(sg.qos != QoSLevel.GENERAL):
                    continue
                for name,record in stat.sg_header[sg].items():
                    members.add(name[:name.find("-")])
                    if(record.is_src_ot):
                        src_ot += 1
                        continue
                    for bs in GV.NET_STATION_CONTROLLER:
                        if( record.is_bs_ot[bs]):
                            bs_ot[bs] += 1
                total_ot = src_ot + sum(bs_ot)
                print(str(tabber)+"{}:size={}, to={}".format(sg,len(members),total_ot))
                tabber.enter()
                print(str(tabber)+"src_ot:{}".format(src_ot))
                print(str(tabber)+"bs_ot:{}".format(bs_ot))
                print(str(tabber)+"members:",members)
                tabber.exit()
            tabber.exit()
ppp()

MAX_N_MEMBER(20)-nQoS-yRSU-NOMA_APR-D1.4-S1
	RCWS:size=0, to=0
		src_ot:0
		bs_ot:[0, 0]
		members: set()
	DYN_SG_0:size=20, to=0
		src_ot:0
		bs_ot:[0, 0]
		members: {'flow_0.15', 'flow_0.23', 'flow_0.54', 'flow_0.58', 'flow_13.4', 'flow_1.9', 'flow_0.31', 'flow_0.62', 'flow_0.19', 'flow_0.39', 'flow_0.27', 'flow_1.6', 'flow_2.37', 'flow_0.18', 'flow_0.21', 'flow_0.46', 'flow_0.14', 'flow_0.33', 'flow_0.57', 'flow_0.35'}
	DYN_SG_1:size=20, to=0
		src_ot:0
		bs_ot:[0, 0]
		members: {'flow_0.49', 'flow_0.64', 'flow_0.17', 'flow_0.50', 'flow_1.7', 'flow_0.48', 'flow_0.42', 'flow_0.25', 'flow_0.53', 'flow_0.52', 'flow_0.61', 'flow_0.13', 'flow_0.22', 'flow_0.20', 'flow_0.16', 'flow_0.36', 'flow_0.29', 'flow_0.43', 'flow_1.3', 'flow_0.56'}
	DYN_SG_2:size=20, to=0
		src_ot:0
		bs_ot:[0, 0]
		members: {'flow_2.38', 'flow_2.64', 'flow_2.69', 'flow_0.37', 'flow_2.68', 'flow_2.55', 'flow_2.46', 'flow_2.78', 'flow_0.26', 'flow_2.56', 'flow_0.30', 'flow_0.45', 'flow_2.35', 'flow_0.60', 'flow_0.34

In [22]:
def ppp():
    tabber = Tabber()
    global statistics
    c = 0
    stat =  statistics[1][0]
    for sg in stat.social_group:
        if(sg.fname!= "DYN_SG_7"):
            continue
        print(len(stat.sg_header[sg]))
        for name,record in stat.sg_header[sg].items():
            if(record.is_src_ot or True in record.is_bs_ot):
                print("===========")
                print(str(stat.interest_config))
                print(name)
                print(record)
                print("===========")
ppp()

1189


In [ ]:
for i in range(100):
    print(ots[0][i])

In [ ]:
for i, statistic in enumerate(statistics):
    print("====={}======".format(i))
    for sg in statistic.social_group:
        ot_count = 0
        for header, record in statistic.sg_header[sg].items():
            if(record.is_src_ot > 0 or
               True in [record.is_bs_ot[i] for i in GV.NET_STATION_CONTROLLER] or
               not True in [record.time_bs_serv[i] > 0 for i in GV.NET_STATION_CONTROLLER] or
               not record.time_veh_serv > 0
               ):
                ot_count += 1
        print("{}: {} => {}".format(sg, ot_count, ot_count / max(len(statistic.sg_header[sg]), 1)))


In [ ]:
class Helper:
    def __init__(self,header,record,bs_index):
        self.name = header
        self.bits = record.bits
        self.time_gen = round(record.at,4)
        self.time_serv = round(record.time_bs_serv[bs_index],4)
        self.time_len =  195-185 if self.time_serv == -1 else round(self.time_serv - self.time_gen,4)
    def __str__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)
    def __repr__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)


In [ ]:
target_header = "flow_9.3-305"
# target_header = "flow_0.58-99"
for statistic in statistics:
    header_in_sg_list = [sg.fname for sg in statistic.social_group   if(target_header in statistic.sg_header[sg])]
    print("In {}".format(header_in_sg_list))
    print("C?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)))
    print("G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.GENERAL)))
    print("C2G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.C2G)))

In [ ]:
for i,statistic in enumerate(statistics):
    for sg in statistic.social_group:
        if target_header in statistic.sg_header[sg]:
            header = statistic.sg_header[sg][target_header]
            print("======={}:{}========".format(i,sg))
            print(header)

In [ ]:
bs_index = 1
social_groups = statistics[0].social_group
print(social_groups)
app_stats_fn = lambda x: statistic.sg_header[social_groups[-1]]
# app_stats_fn = lambda x: statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)
# app_stats_fn = lambda x: x.sg_header[SocialGroup.RCWS]

In [ ]:
bs_serv_list = [[] for _ in statistics]
bs_time_gap = [{} for _ in statistics]
for i, statistic in enumerate(statistics):
    for header, record in app_stats_fn(statistic).items():
        if(len(record.time_bs_txq[bs_index]) > 0):
            bs_serv_list[i].append(Helper(header, record, bs_index))
    bs_serv_list[i].sort(key=lambda x : x.time_gen*1000000+x.time_serv)
    for helper in sorted(bs_serv_list[i],key=lambda x: (x.time_serv * 1000) * 10000000 + x.bits):
        at = str(helper.time_gen)
        to = str(helper.time_serv)
        if(to not in bs_time_gap[i]):
             bs_time_gap[i][to] = {"need": 0, "give": 0}
        if(at not in bs_time_gap[i]):
             bs_time_gap[i][at] = {"need": 0, "give": 0}
        bs_time_gap[i][to]["give"]+=helper.bits
        bs_time_gap[i][at]["need"]+=helper.bits
    
   


In [ ]:
for v in bs_serv_list[0]:
    print(v)

In [ ]:
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    data = []
    for i in range(2):
        if(time in bs_time_gap[i]):
            data.append(bs_time_gap[i][time]["need"])
            data.append(bs_time_gap[i][time]["give"])
        else:
            data.append(0)
            data.append(0)
    if(not sum(data)==0):
        print(time, data)


In [ ]:

remain = [0 for _ in statistics]
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    for i in range(2):
        if(time in bs_time_gap[i]):
            helper = bs_time_gap[i][time]
            remain[i] += (helper["need"] - helper["give"])
    if(not sum(remain)==0):
        print(time, remain, remain[0]-remain[1])
